Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.

![Maze](Maze.jpg)



The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the form of a table.
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 

Note 1: You must follow the problem formulation discussed in the class. Abstract classe for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: The code should be written in a single jupyter notebook file.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

In [2]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [3]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [4]:
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
  
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [5]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

In [6]:
MAZE_ROWS = 16
MAZE_COLUMNS = 24
ACTIONS = {'UP', 'LEFT', 'RIGHT', 'DOWN'}
    
class Maze(Problem):
    
    shaded_cells = [(14,9), (13,10), (12,10), (11,10), (10,10), (10,9), (9,9), (8,9), (7,9), (6,9), (11,12), (11,13), (10,12), (10,13)]

    def __init__(self, initial=None, goal=None):
        Problem.__init__(self, initial=initial, goal=goal)

    def h(self, node):
        # This is the heuristics, currently implementing Euclidian distance
        x1, y1 = node.state
        x2, y2 = self.goal
        return abs(x1 - x2) + abs(y1 - y2)

    def action_cost(self, s,a,s1):
        if a in {'LEFT', 'RIGHT'}:
            return 10
        elif a in {'UP', 'DOWN'}:
            return 1

    def result(self, state, action):
        x, y = state
        if action == 'UP' and x < MAZE_ROWS - 2:
            x_new = x + 1
        elif action == 'DOWN' and x > 0:
            x_new = x - 1
        else:
            x_new = x

        if action == 'LEFT' and y > 0:
            y_new = y - 1
        elif action == 'RIGHT' and y < MAZE_COLUMNS - 2:
            y_new = y + 1
        else:
            y_new = y

        if not self.is_shaded(x_new, y_new):
            return (x_new, y_new)
        else:
            return (x, y)

    def actions(self, state):
        x, y = state
        valid_actions = set()

        if x < MAZE_ROWS - 2 and not self.is_shaded(x + 1, y):
            valid_actions.add('UP')
        if x > 0 and not self.is_shaded(x - 1, y):
            valid_actions.add('DOWN')
        if y < MAZE_COLUMNS - 2 and not self.is_shaded(x, y + 1):
            valid_actions.add('RIGHT')
        if y > 0 and not self.is_shaded(x, y - 1):
            valid_actions.add('LEFT')

        return valid_actions

    def is_shaded(self, x, y):
        return (x, y) in self.shaded_cells
    
    def goal_test(self, state):
        return state == self.goal

### Testing the model

In [7]:
#Testing Maze class
#Note that we perform additional tests to validate the modeling

#An instance of Maze problem; initializa the Maze class with appropriate values
m1 = Maze(initial = None, goal = None)

# Testing the methods in Maze class
assert m1.result((4,12),'UP') == (5,12)
assert m1.action_cost((4,12),'UP',(5,12)) == 1
assert m1.actions((4,12)) == {'UP', 'LEFT', 'RIGHT', 'DOWN'}

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

In [8]:
#Your code for breadth-first search

#Keep track of the blocks of maze
class Block_Position:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
def solveMaze_bfs(maze, goal, start):
    
    #Set move direction for DOWN and UP
    adj_cell_x = [-1, 0, 0, 1]
    
    #Set move directions for lEFT and RIGHT
    adj_cell_y = [0, -1, 1, 0]
    
    #Maintain visited nodes
    visited_blocks = [[False for i in range(MAZE_COLUMNS)] for j in range(MAZE_ROWS)]
    visited_blocks[start.x][start.y] = True
    
    #Set the starting point
    current_block = Node(maze.initial)
    
    if maze.goal_test(current_block.state):
        return current_block
    
    frontier = deque([node])
    explored = set()
    
    action_cost = 0
    cells = 4
    
    while frontier:
        current_block = frontier.popleft()  # Dequeue the front cell
        current_pos = current_block.pos
        
        explored.add(current_block.state)
        
        #Check for goal state
        if current_pos.x == goal.x and current_pos.y == goal.y:
            print("Algorithm used = BFS")
            print("Path found!")
            return current_block.path_cost
             
        if current_block not in visited_blocks:
            visited_blocks[current_pos.x][current_pos.y] = True
            
        x_pos = current_pos.x
        y_pos = current_pos.y
        
        for i in range(cells):
            #Check for top boundary (wall)
            if x_pos == MAZE_ROWS - 1 and adj_cell_x[i] == 1:
                x_pos = current_pos.x
                y_pos = current_pos.y + adj_cell_y[i]
            
            #Check for bottopn boundary (wall)
            if x_pos == 0 and adj_cell_x[i] == -1:
                x_pos = current_pos.x
                y_pos = current_pos.y + adj_cell_y[i]
            
            #Check for right boundary (wall)
            if y_pos == MAZE_COLUMNS - 1 and adj_cell_y[i] == 1:
                x_pos = current_pos.x + adj_cell_x[i]
                y_pos = current_pos.y
                
            #Check for left boundary (wall)
            if y_pos == 0 and adj_cell_y[i] == -1:
                x_pos = current_pos.x + adj_cell_x[i]
                y_pos = current_pos.y
            
            else:
                x_pos = current_pos.x + adj_cell_x[i]
                y_pos = current_pos.y + adj_cell_y[i]
            
            for child in current_block.expand(maze):
                
                if x_pos < MAZE_COLUMNS and y_pos < MAZE_ROWS and x_pos >= 0 and y_pos >= 0:
                    
                    #Check if next cell is empty cell
                    if maze[x_pos][y_pos] == 1:

                        if child.state not in explored and child not in frontier:
                            if maze.goal_test(child.state):
                                return child
                        
                            #Add action cost to move DOWN OR UP
                            if adj_cell_x[i] == -1 or 1:
                                action_cost = 1

                            #Add action cost to move LEFT OR RIGHT
                            if adj_cell_y[i] == -1 or 1: 
                                 action_cost = 10

                            child = Node(Block_Position(x_pos, y_pos), current_block.action_cost + action_cost)
                            
                            visited_blocks [x_pos][y_pos] = True
                            
                            frontier.append(child)
    return None
     

In [9]:
from collections import deque

def depth_first_search(problem):
    frontier = deque()
    initial_node = Node(problem.initial)
    frontier.append(initial_node)

    explored = set()

    expanded_nodes = 0
    max_frontier_size = 0

    while frontier:
        max_frontier_size = max(max_frontier_size, len(frontier))
        node = frontier.pop()
        expanded_nodes += 1
        explored.add(node.state)

        if problem.goal_test(node.state):
            return node, expanded_nodes, max_frontier_size

        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)

            if child_state not in explored and not any(node.state == child_state for node in frontier):
                child_node = Node(child_state, node, action)
                frontier.append(child_node)

    return None, expanded_nodes, max_frontier_size 

start_state = (8, 10)
goal_state = (11, 9)

maze_problem = Maze(initial=start_state, goal=goal_state)
solution_node, expanded_nodes, max_frontier_size = depth_first_search(maze_problem)

if solution_node:
    path = []
    path_cost = 0
    while solution_node.parent is not None:
        path.insert(0, solution_node.action)
        path_cost += maze_problem.action_cost(s=solution_node.parent.state,a = solution_node.action,
                                             s1=solution_node.state)
        solution_node = solution_node.parent
    print("Solution path:", path)
    print("Expanded nodes:", expanded_nodes)
    print("Maximum frontier size:", max_frontier_size)
    print("Path cost:", path_cost)
else:
    print("No solution found")

Solution path: ['DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'UP', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'UP', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'UP', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'UP', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'UP', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'RIGHT']
Expanded nodes: 240
Maximum frontier size: 85
Path cost: 529


In [10]:
#Your code for  iterative Deepening search with cycle-check

In [72]:
def memoize(fn, slot=None):
    """Memoization decorator for a function with a single argument."""
    if slot is None:
        slot = fn.__name__

    def decorator(self, *args, **kwargs):
        if hasattr(self, slot):
            return getattr(self, slot)
        else:
            result = fn(self, *args, **kwargs)
            setattr(self, slot, result)
            return result

    return decorator


In [73]:
# sample code for best first graph seach

def best_first_graph_search(problem,heuristic, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    frontier = PriorityQueue()
    
    f = memoize(heuristic, 'f')
    
    initial_node = Node(problem.initial)
    
    frontier.add(initial_node)
    
    # Define the heuristic function f to use the h function from the Maze class
    #f = lambda node: problem.h(node)
    
    explored = set()
    
    expanded_nodes = 0
    max_frontier_size = 0
    
    
    while frontier:
        max_frontier_size = max(max_frontier_size, len(frontier))
        node = frontier.pop()
        expanded_nodes += 1
        explored.add(node.state)
        
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node, expanded_nodes, max_frontier_size
        explored.add(node.state)
        
        
        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)
            if child_state not in explored and not frontier.contains_state(child_state):
                child_node = Node(child_state, node, action)
                f_value = f(child_node)  # Calculate the f value using the heuristic
                frontier.add(child_node)

    return None  # No solution found

In [83]:
start_state = (8, 10)
goal_state = (11, 9)

maze_problem = Maze(initial=start_state, goal=goal_state)

# Perform the search with the heuristic
solution_node, expanded_nodes, max_frontier_size = best_first_graph_search(maze_problem,lambda node: maze_problem.h(node))  # Remove expanded_nodes and max_frontier_size

if solution_node:
    path = []
    path_cost = 0
    while solution_node.parent is not None:
        path.insert(0, solution_node.action)
        path_cost += maze_problem.action_cost(s=solution_node.parent.state,
                                              a = solution_node.action,
                                             s1=solution_node.state)
        solution_node = solution_node.parent
    print("Solution path:", path)
    print("Expanded nodes:", expanded_nodes)
    print("Maximum frontier size:", max_frontier_size)
    print("Path cost:", path_cost)
else:
    print("No solution found")



Solution path: ['DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'UP', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'RIGHT']
Expanded nodes: 1969
Maximum frontier size: 629
Path cost: 473


In [84]:
def uniform_cost_search(problem, display=False):
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda node: node.path_cost, display)



In [85]:
solution_node, expanded_nodes, max_frontier_size = uniform_cost_search(maze_problem)

if solution_node:
    path = []
    path_cost = 0
    while solution_node.parent is not None:
        path.insert(0, solution_node.action)
        path_cost += maze_problem.action_cost(s=solution_node.parent.state,a = solution_node.action,
                                             s1=solution_node.state)
        solution_node = solution_node.parent
    print("Solution path:", path)
    print("Expanded nodes:", expanded_nodes)
    print("Maximum frontier size:", max_frontier_size)
    print("Path cost:", path_cost)
else:
    print("No solution found")

Solution path: ['DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'UP', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'RIGHT']
Expanded nodes: 1969
Maximum frontier size: 629
Path cost: 473


In [86]:
def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

In [87]:
solution_node, expanded_nodes, max_frontier_size = astar_search(maze_problem)

if solution_node:
    path = []
    path_cost = 0
    while solution_node.parent is not None:
        path.insert(0, solution_node.action)
        path_cost += maze_problem.action_cost(s=solution_node.parent.state,a = solution_node.action,
                                             s1=solution_node.state)
        solution_node = solution_node.parent
    print("Solution path:", path)
    print("Expanded nodes:", expanded_nodes)
    print("Maximum frontier size:", max_frontier_size)
    print("Path cost:", path_cost)
else:
    print("No solution found")

Solution path: ['DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'UP', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'RIGHT']
Expanded nodes: 1969
Maximum frontier size: 629
Path cost: 473


In [66]:
#Your code for greedy-best first search

greedy_best_first_graph_search = best_first_graph_search


# Greedy best-first search is accomplished by specifying f(n) = h(n).

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the following table.


 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost                  |
|:---------------------|:-------------------------|:-----------------------|:----------
| Breadth-First Search |                          |                        |
| DFS with cycle check |                          |                        |
| IDS with cycle check |                          |                        |
| UCS                  |                          |                        |
| A* Search            |                          |                        |
| GBFS                 |                          |                        |

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.

